In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization
from matplotlib import pyplot as plt # data visualization

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv
/kaggle/input/cleaned-data/Mushroom train_data__Cleaned.csv
/kaggle/input/cleaned-data/Mushroom test_data__Cleaned.csv
/kaggle/input/tx-pipeline/tx_pipe.pkl


In [7]:
mdf = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
mdf_test = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv")

## **Knowing the Data:~**

In [ ]:
mdf.set_index('id', inplace=True)
mdf_test.set_index('id', inplace=True)

In [ ]:
mdf

In [ ]:
mdf_test

In [ ]:
cmbd_data = pd.concat([mdf,mdf_test])
cmbd_data.sample(10)

In [ ]:
submission

In [ ]:
mdf.sample(10)

In [ ]:
display(mdf.columns)
display(mdf_test.columns)

In [ ]:
mdf.columns = mdf.columns.str.replace("-", "_", regex=True)
mdf.rename(columns={"class":"e_label"}, inplace=True)

mdf_test.columns = mdf_test.columns.str.replace("-", "_", regex=True)

In [ ]:
mdf.isnull().sum()

In [ ]:
mdf_test.isnull().sum()

In [ ]:
mdf[mdf.duplicated()]

In [ ]:
mdf_test[mdf_test.duplicated()]

#### //- SUMMARY from above:
* ##### The dataset has **22 columns**, with "id" column set as index. So effectively 21.
* ##### There are **3116945 rows. No duplicate rows** in the train dataset. 
* ##### Test dataset has **2077964 rows, and 20 cols**. No duplicates here as well.
* ##### Some of the columns have **a lot of null** values in both train & test data..
* ##### **Target column name is "e_label"** meaning Edibility-label.

### **Details of the Numerical Columns:~**

In [ ]:
mdf.describe()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
fig.suptitle('Box Plots of --')

# cap_diameter
sns.boxplot(ax=axes[0], x=mdf["cap_diameter"])
axes[0].set_title("Cap Diameter")

# stem_height
sns.boxplot(ax=axes[1], x=mdf["stem_height"])
axes[1].set_title("Stem Height")

# stem_width
sns.boxplot(ax=axes[2], x=mdf["stem_width"])
axes[2].set_title("Stem Width")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 4), sharey=True)
fig.suptitle('Histplot of --')

# cap_diameter
sns.histplot(x=mdf["cap_diameter"], ax=axes[0],kde=True, bins=15)
axes[0].set_title("Cap Diameter")

# stem_height
sns.histplot(x=mdf["stem_height"], ax=axes[1],kde=True, bins=15)
axes[1].set_title("Stem Height")

# stem_width
sns.histplot(x=mdf["stem_width"], ax=axes[2],kde=True, bins=15)
axes[2].set_title("Stem Width")

### **Details of the Categorical Columns:~**

In [ ]:
mdf.describe(include="object")

In [ ]:
display(mdf.e_label.value_counts())
mdf.e_label.value_counts().plot(kind="pie", title="Proportion of e & p", ylabel="",
                                y=mdf.e_label.value_counts(), figsize=(4,4),
                                autopct="%1.01f%%", explode=(0.01,0.02))

In [ ]:
mdf.shape

In [ ]:
mdf.cap_shape.value_counts()

In [ ]:
mdf.cap_shape.unique()

##### In cap_shape there are some garbage values, probably are wringly imputed. Let's check how much data are such.

In [ ]:
mdf[mdf.cap_shape.isin(list("fxpbocsdenwkltgzaruyimh")+[np.nan])].shape

##### So there are **(3116945-3116890) = 55** such data. We can remove these.

In [ ]:
mdf = mdf[mdf.cap_shape.isin(list("fxpbocsdenwkltgzaruyimh")+[np.nan])]
sns.catplot(mdf, kind="count", x ="cap_shape", height=3, aspect=3)

##### Proportion wise only 7 entries like **f,x,p,b,o,c,s have visible spread over the cap_shape series.**

In [ ]:
mdf.shape

In [ ]:
mdf.cap_surface.value_counts()

##### The same is for cap_surface too. So have done the same operation and dropped those garbage entries.

In [ ]:
mdf = mdf[mdf.cap_surface.isin(list("shyltegdiwkfnroauzpbmxc")+[np.nan])]
sns.catplot(mdf, kind="count", x ="cap_surface", height=3, aspect=3)

In [ ]:
mdf.shape #73 rows dropped in this process.

In [ ]:
mdf.cap_color.value_counts()

In [ ]:
mdf = mdf[mdf.cap_color.isin(list("uobgwneyrpklihdsafcxmzt")+[np.nan])]
sns.catplot(mdf, kind="count", x ="cap_color", height=3, aspect=3)

In [ ]:
mdf.shape #72 records removed

In [ ]:
mdf.does_bruise_or_bleed.value_counts().head(10)

##### The column name suggests that it should be a Binary column. Checked on the official website as well, which confirms the same as well. Hence kept only t & f, as in True and False.

In [ ]:
mdf = mdf[mdf.does_bruise_or_bleed.isin(list("tf"))]
sns.catplot(data = mdf, x='does_bruise_or_bleed', kind="count", height=3, aspect=1)

In [ ]:
mdf.shape #117 records dropped

In [ ]:
# list('abcdefghijklmnopqrstuvwxyz')
atoz = [chr(i) for i in range(ord('a'), ord('z')+1)]

def non_alpha_categories_removal(df, col, atoz):
    len_before = df.shape[0]
    df = df[df[col].isin(atoz+[np.nan])]
    len_after = df.shape[0]
    print(f"'{col}' processed. {len_before-len_after} records removed")
    sns.catplot(df, kind="count", x =col, height=2, aspect=3)
    return df


for col in ['gill_attachment', 'gill_spacing', 'gill_color',
            'stem_root', 'stem_surface', 'stem_color', 'veil_type', 
            'veil_color', 'has_ring', 'ring_type', 'spore_print_color', 'habitat']:
    mdf = non_alpha_categories_removal(mdf,col, atoz)

In [ ]:
mdf.shape

In [ ]:
display(mdf.season.value_counts())
mdf.season.value_counts().plot(figsize=(7,4), kind="pie", y=mdf.season.value_counts(),
                              autopct = "%1.01f%%", title="Distribution of Season", ylabel="")

#### //- SUMMARY from above:
1. ##### The **numerical columns have a lot of outliers.** Stem height column is normally distributed, while the others are right-skewed.
2. ##### The **target column is almost equally distributed** among Edible and Poisonous
3. ##### Most of the other **categorical columns had some garbage values** like numerical records or class name and so. So, have removed these garbage records **(about 700 rows dropped).** Still, *not all the alphabetical values are having significant spreads across respective columns.* So need to verify their correctness to decide whether to keep them or not.
4. ##### About **4 Mushrooms in every 5 don't have bruising or bleeding.**
5. ##### There are **4 different seasons** spreading across the dataset, which are most probably **"Autumn", "Summer", "Winter", and "Spring".** --> Spring occurs the least among all.

## **Data Cleaning & Processing:~**

In [ ]:
temp = mdf.copy()

In [ ]:
# mdf = temp

In [ ]:
mdf

### **Outlier Removal:**

#### Previuosly we had concluded that the numerical columns had good amount of outliers. Let's try to fix those now.

In [ ]:
num_cols = ['cap_diameter', 'stem_height', 'stem_width']

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 6), sharey=True)
fig.suptitle('-- Spread in Nuemrical columns -- ')
color_map = ['#ec407a','#16a085', '#7986cb']

for n,col in enumerate(num_cols):
    sns.boxplot(ax=axes[n][0], x=mdf[col], color=color_map[n])
    axes[n][0].set_title(f"Box Plot of {col}", backgroundcolor='lightgrey')
    axes[n][0].set_xlabel("")
    
    sns.violinplot(ax=axes[n][1], x=mdf[col],  color=color_map[n])
    axes[n][1].set_title(f"Violin Plot of {col}", backgroundcolor='lightgrey')
    axes[n][1].set_xlabel("")
    
fig.tight_layout()

In [ ]:
#Outlier Removal using IQR Method

for col in num_cols:
    Q1 = mdf[col].quantile(.25)
    Q3 = mdf[col].quantile(.75)
    IQR = Q3-Q1
    
    LL = Q1-1.5*IQR
    UL = Q3+1.5*IQR
    print(f"{mdf[(mdf[col]<LL) | (mdf[col]>UL)].shape[0]} rows are removed:")
    mdf = mdf[(mdf[col]>=LL) & (mdf[col]<=UL)]


In [ ]:
mdf.shape

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 6), sharey=True)
fig.suptitle('-- Spread in Nuemrical columns -- ')
color_map = ['#ec407a','#16a085', '#7986cb']

for n,col in enumerate(num_cols):
    sns.boxplot(ax=axes[n][0], x=mdf[col], color=color_map[n])
    axes[n][0].set_title(f"Box Plot of {col}", backgroundcolor='lightgrey')
    axes[n][0].set_xlabel("")
    
    sns.violinplot(ax=axes[n][1], x=mdf[col],  color=color_map[n])
    axes[n][1].set_title(f"Violin Plot of {col}", backgroundcolor='lightgrey')
    axes[n][1].set_xlabel("")
    
fig.tight_layout()

### **Handling Null/ Missing Values:**

In [ ]:
mdf.isnull().mean().round(4) * 100

In [ ]:
(mdf.isnull().mean().round(4) * 100).plot.bar(title="Percentage of null values in each columns",figsize=(12,3), color='brown')

#### So, from the above diagram, it's clear that a few columns like `"stem_root"`, `"stem_surface"`, `"veil_type"`, `"veil_color"`, and `"spore_print_color"` **has 60-96% data missing.** 

#### And, **we should not impute these many records**. 
> Logically, when there are more than 50% of data missing and there's no known reason of this, we should not fill in any values, and drop such columns. Because in that case more than half of things we are doing don't have any conclusive evidence of why we are doing.
#### So we will remove this columns.


In [ ]:
# Let's drop any columns that has more than 50% null value
threshold_na = len(mdf) * 0.50

# Drop columns with more than 50% missing values
mdf = mdf.dropna(thresh=threshold_na, axis=1)

In [ ]:
mdf.sample(2)

#### Four other columns namely, `"cap_surface"`, `"gill_attachment"`, `"gill_spacing"` & `"ring_type"` has visible null amount. For now we can impute all missing values in these columns as "unk", as in "Unknown". 
#### Later on these columns can be judged based on correlation with target. 

In [ ]:
mdf[["cap_surface", "gill_attachment", "gill_spacing", "ring_type"]]=mdf[["cap_surface", "gill_attachment", "gill_spacing", "ring_type"]].fillna("unk")

In [ ]:
mdf[["cap_surface", "gill_attachment", "gill_spacing", "ring_type"]].isnull().sum()

In [ ]:
mdf.ring_type.value_counts()

In [ ]:
mdf.isnull().sum()

#### As a result of the above operation, we have successfully handled most of the null values. Rest of the columns have very few null entries (near to 0%). We can drop these.

In [ ]:
mdf.dropna(inplace=True)
mdf.isnull().sum()

### **Removing unwanted/rare data:**

In [ ]:
mdf.nunique()

#### Earlier we had seen than all these categorical columns have many unique values in them. And most of those have very low spread in the dataset. 
#### The above output shows that apart from the target column, three numerical columns & 2 other categorical columns, all other have more than 20 unique values. This much variation will confuse both us and our ML algo. 
####  

#### What are we gonna do then? 
> Let's consider only those unique values that have atleast 5% spread in the respective columns. The rest of them might have been wrongly imputed or are rare. We can change those to `Other`

In [ ]:
mdf.cap_shape.value_counts()/mdf.cap_shape.count()*100 #will consider till cap_shape='b', rest-->'other'

#mdf.cap_shape.value_counts(normalize=True) yields the same

In [ ]:
percent_counts = mdf.cap_shape.value_counts()/mdf.cap_shape.count()*100
idx = percent_counts[percent_counts<5].index
print(idx)

mdf.loc[mdf.cap_shape.isin(idx), 'cap_shape'] = "other"
print(mdf.cap_shape.value_counts()/mdf.cap_shape.count()*100)  

#### This makes the change as we indended for. But **if in any case, the percentage of 'other' too is less than 5%,** we can drop them.

In [ ]:
for col in ['cap_surface', 'cap_color', 'gill_attachment', 'gill_spacing', 'gill_color',
            'stem_color', 'has_ring', 'ring_type', 'habitat']:

    percent_counts = mdf[col].value_counts()/mdf[col].count()*100
    idx = percent_counts[percent_counts<5].index

    if percent_counts[percent_counts<5].sum()>=5:
        mdf.loc[mdf[col].isin(idx), col] = "other"
        print(f"Rare categories in '{col}' column are classed as 'other'")
    
    else:
        mdf = mdf.loc[~mdf[col].isin(idx)]
        print(f"Rare categories in '{col}' column are dropped")    
    

In [ ]:
mdf.nunique()

#### Here's information on most frequest values of each categorical columns

* **cap-shape (n):** bell=b, convex=x, flat=f, sunken=s,
* **cap-color (n):** brown=n, gray=g, white=w, yellow=y, orange=o, red=e,
* **does-bruise-or-bleed (n):** true=t,false=f,
* **gill-spacing (n):** close=c, distant=d,
* **gill-color (n):**  brown=n, white=w, gray=g, pink=p, yellow=y, orange=o,
* **stem-color (n):**  brown=n, white=w, yellow=y, 
* **has-ring (n):** true=t, false=f,
* **ring-type (n):** flaring=f,
* **habitat (n):** grasses=g, leaves=l, meadows=m, woods=d,
* **season (n):** spring=s, summer=u, autumn=a, winter=w

In [ ]:
# Category name maping 

mdf.cap_shape = mdf.cap_shape.replace({'b':"bell", 'x':"convex", 'f':"flat", 's':"sunken"})
mdf.cap_color = mdf.cap_color.replace({'n':"brown", 'g':"gray", 'w':"white", 'y':"yellow", 'o':"orange", 'e':"red"})
mdf.gill_spacing = mdf.gill_spacing.replace({'c':"close", 'd':"distant"})
mdf.gill_color = mdf.gill_color.replace({'n':"brown", 'g':"gray", 'w':"white", 'p':"pink", 'y':"yellow", 'o':"orange"})
mdf.stem_color = mdf.stem_color.replace({'n':"brown", 'w':"white", 'y':"yellow"})
mdf.ring_type = mdf.ring_type.replace({'f':"flaring"})
mdf.habitat = mdf.habitat.replace({'g':"grasses", 'l':"leaves", 'm':"meadows", 'd':"woods"})
mdf.season = mdf.season.replace({'s':"spring", 'u':"summer", 'w':"winter", 'a':"autumn"})

In [ ]:
mdf.sample(2)

In [ ]:
categorical_cols= ['cap_shape', 'cap_surface', 'cap_color', 'gill_attachment', 'gill_spacing', 'gill_color', 
                   'stem_color', 'has_ring', 'ring_type', 'does_bruise_or_bleed', 'habitat', 'season' ]

In [ ]:
for col in categorical_cols:
    print(mdf[col].value_counts(normalize=True))
    print("============================================")

In [ ]:
cat_cols = np.reshape(categorical_cols, (4,3))

# for col in categorical_cols:
f, axs = plt.subplots(4, 3, figsize=(18, 15))

for n, col in enumerate(cat_cols):
    sns.countplot(ax=axs[n,0], data=mdf, x =col[0])
    axs[n,0].set_title(f"Freq. of each category in {col[0]}")
    axs[n,0].set_xlabel("")
    axs[n,0].set_ylabel("")
    
    sns.countplot(ax=axs[n,1], data=mdf, x =col[1])
    axs[n,1].set_title(f"Freq. of each category in {col[1]}")
    axs[n,1].set_xlabel("")
    axs[n,1].set_ylabel("")
    
    sns.countplot(ax=axs[n,2], data=mdf, x =col[2])
    axs[n,2].set_title(f"Freq. of each category in {col[2]}")
    axs[n,2].set_xlabel("")
    axs[n,2].set_ylabel("")

f.tight_layout()

In [ ]:
mdf.shape

#### //- SUMMARY from above:
1. ##### Outliers from the numerical columns are removed. **Around 2.5 lac records (~ 8% of whole) dropped.**
2. ##### **"stem_root", "stem_surface", "veil_type", "veil_color", and "spore_print_color"** columns had more than 50% null values. Hence those columns were *dropped.* 
3. ##### Null values in **"cap_surface", "gill_attachment", "gill_spacing" & "ring_type"** columns were *imputed with "unk" as in "Unknown".*
4. ##### In other categorical columns... *only those categories are kept that had a **spread of 5% or more** in the respective columns. If not, all are combined to form a new category as "Other".*
5. ##### **Categories were replaced with their more convenient names** with reference from original website.
6. ##### As of now there are **16 columns** *(1 target column, 3 Numerical ones, 3 binary & rest of those are nominal ones)*. Around **27.5 lac rows present.**

### **Bi-variate & Multivariate Analysis:**

In [ ]:
mdf.sample(5)

In [ ]:
sns.kdeplot(data =mdf, fill=True, x='cap_diameter', hue='e_label')
print("Mushrooms with less cap diameter have a higher chance of being Poisonous.")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='cap_diameter', hue='season', height=3, aspect=2)
print("Mushrooms is Spring season are mostly smaller in cap diameter.")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='cap_diameter', hue='habitat', height=3, aspect=2)

In [ ]:
sns.displot(data =mdf, kind='kde', x='stem_height', hue='e_label', height=3, aspect=2)
print("Mostly mushrooms' stems are between 4-8 cm tall, with equal probability of e&p")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='stem_height', hue='season', height=3, aspect=2)
print("Stem heights are almost constant across different seasons, and edibility.")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='stem_height', hue='habitat', height=3, aspect=2)

In [ ]:
sns.displot(data =mdf, x='stem_width', hue='e_label', height=3, aspect=2)

In [ ]:
sns.displot(data =mdf, kind='kde', x='stem_width', hue='e_label', height=3, aspect=2)
print("The narrower Mushrooms tend to be more poisonous.")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='stem_width', hue='season', height=3, aspect=2)
print("Mushrooms growing in Spring season are comparatively much narrower.")

In [ ]:
sns.catplot(kind='box', data =mdf, x='e_label', y='stem_width', hue='habitat', height=3, aspect=2)

In [ ]:
cat_cols = np.reshape(categorical_cols, (4,3))

# for col in categorical_cols:
f, axs = plt.subplots(4, 3, figsize=(15, 15))
f.suptitle('-- "e_label" proportion in each Categorical columns -- ', fontsize=17)
c_palette={'e':'#18ac8f', 'p':'#e5306d'}

for n, col in enumerate(cat_cols):
    sns.histplot(ax=axs[n,0], data=mdf, x =col[0], hue='e_label', multiple='stack', palette=c_palette)
    sns.histplot(ax=axs[n,1], data=mdf, x =col[1], hue='e_label', multiple='fill', palette=c_palette)
    sns.histplot(ax=axs[n,2], data=mdf, x =col[2], hue='e_label', multiple='fill', palette=c_palette)

    
print("-- Be remindful of these before eating or buying Mushrooms --","\n============================================")    
print("* Mushrooms with bell-shaped cap are more like to be poisonous, others have about 50% possibility.")
print("* Mushrooms with red or orange coloured cap should better be avoided.")
print("* White stem Mushrooms can be eaten, but be cautious that those ain't towards the yellowish side.")
print("* Mushrooms growing in grass or meadows are more likely to be poisonous. The ones in woods & leaves are a better choice.")
print("* You can better eat mushrooms of Winters. Otherwise there 50-50 between e & p.")

##### The target column is well-balanced, as we have seen at the begining. So we don't need to bother about Sampling. So that's it with EDA. 

#### Let's sum up the main steps we have executed, as the same need to be done for the test data as well.
1. **Removing the non-alphabetical (noisy) categories** from the Categorical columns
2. **Keep only t&f** for `does_bruise_or_bleed` column
3. **Outlier removal**
4. **Null Value Handling~**
    * **Dropping columns with more than 50% null entries.**
    * **Impute 'unk' for other columns** with 10-50% null. 
    * **Drop rows for NA entries in other cols**
5. **Removing unwanted/rare data**

In [ ]:
#1 Removing the non-alphabetical (noisy) categories

for col in ['cap_shape', 'cap_surface', 'cap_color', 'gill_attachment', 'gill_spacing',
            'gill_color', 'stem_root', 'stem_surface', 'stem_color', 'veil_type', 
            'veil_color', 'has_ring', 'ring_type', 'spore_print_color', 'habitat']:

    mdf_test = non_alpha_categories_removal(mdf_test, col, atoz)

In [ ]:
mdf_test.season.value_counts().plot(figsize=(5,3), kind="pie", y=mdf_test.season.value_counts(),
                              autopct = "%1.01f%%", title="Distribution of Season", ylabel="")
plt.show()

In [ ]:
#2 Keep only t&f for does_bruise_or_bleed

mdf_test = mdf_test[mdf_test.does_bruise_or_bleed.isin(list("tf"))]

mdf_test.does_bruise_or_bleed.value_counts().plot(figsize=(5,3), kind="pie", y=mdf_test.does_bruise_or_bleed.value_counts(),
                              autopct = "%1.01f%%", title="Home many with Bruises/bleedings?", ylabel="")
plt.show()

In [ ]:
#3 Outlier Removal using IQR Method

for col in num_cols:
    Q1 = mdf_test[col].quantile(.25)
    Q3 = mdf_test[col].quantile(.75)
    IQR = Q3-Q1
    
    LL = Q1-1.5*IQR
    UL = Q3+1.5*IQR
    print(f"{mdf_test[(mdf_test[col]<LL) | (mdf_test[col]>UL)].shape[0]} rows are removed:")
    mdf_test = mdf_test[(mdf_test[col]>=LL) & (mdf_test[col]<=UL)]

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 6), sharey=True)
fig.suptitle('-- Spread in Nuemrical columns -- ')
color_map = ['#7d57b0','#de513b', '#7dd137']

for n,col in enumerate(num_cols):
    sns.boxplot(ax=axes[n][0], x=mdf_test[col], color=color_map[n])
    axes[n][0].set_title(f"Box Plot of {col}", backgroundcolor='lightgrey')
    axes[n][0].set_xlabel("")
    
    sns.violinplot(ax=axes[n][1], x=mdf_test[col],  color=color_map[n])
    axes[n][1].set_title(f"Violin Plot of {col}", backgroundcolor='lightgrey')
    axes[n][1].set_xlabel("")
    
fig.tight_layout()

In [ ]:
#4 Null Handling

(mdf_test.isnull().mean().round(4) * 100).plot.bar(title="Percentage of null values in each columns",figsize=(12,3), color='brown')

In [ ]:
threshold_na = len(mdf_test) * 0.50

# Drop columns with more than 50% missing values
mdf_test = mdf_test.dropna(thresh=threshold_na, axis=1)

In [ ]:
mdf_test.columns

In [ ]:
mdf_test[["cap_surface", "gill_attachment", "gill_spacing", "ring_type"]]=mdf_test[["cap_surface", "gill_attachment", "gill_spacing", "ring_type"]].fillna("unk")

In [ ]:
mdf_test.dropna(inplace=True)
mdf_test.isnull().sum()

In [ ]:
mdf_test.shape

In [ ]:
#5 Removing unwanted/rare data
## Instead using replace, as earlier, we will use map now. 
## This would make any other categories mapped to null, which we can later be changed to "Other"

mdf_test.cap_shape = mdf_test.cap_shape.map({'b':"bell", 'x':"convex", 'f':"flat", 's':"sunken"})
mdf_test.cap_color = mdf_test.cap_color.map({'n':"brown", 'g':"gray", 'w':"white", 'y':"yellow", 'o':"orange", 'e':"red"})
mdf_test.gill_spacing = mdf_test.gill_spacing.map({'c':"close", 'd':"distant", "unk":"unk"})
mdf_test.gill_color = mdf_test.gill_color.map({'n':"brown", 'g':"gray", 'w':"white", 'p':"pink", 'y':"yellow", 'o':"orange"})
mdf_test.stem_color = mdf_test.stem_color.map({'n':"brown", 'w':"white", 'y':"yellow"})
mdf_test.ring_type = mdf_test.ring_type.map({'f':"flaring"})
mdf_test.habitat = mdf_test.habitat.map({'g':"grasses", 'l':"leaves", 'm':"meadows", 'd':"woods"})
mdf_test.season = mdf_test.season.map({'s':"spring", 'u':"summer", 'w':"winter", 'a':"autumn"})

#for other columns (matched with train data)
mdf_test.cap_surface = np.where(mdf_test.cap_surface.isin(['s','h','y','t','g','d','unk']),mdf_test.cap_surface, "other")
mdf_test = mdf_test[mdf_test.gill_attachment.isin(['s','a','x','d','e','p','unk'])]
mdf_test = mdf_test[mdf_test.has_ring.isin(['t','f'])]

mdf_test.dropna(subset = ['gill_spacing'], inplace=True)  #'other' is not there in gill_spacing for mdf, hence NAs ain't kept to be filled.
mdf_test.fillna("other", inplace=True)

In [ ]:
mdf_test.shape

In [ ]:
for col in categorical_cols:
    print(mdf_test[col].value_counts(normalize=True))
    print("============================================")

In [ ]:
print(mdf.nunique(),"\n=======================")
print(mdf_test.nunique())

In [ ]:
cat_cols = np.reshape(categorical_cols, (4,3))

# for col in categorical_cols:
f, axs = plt.subplots(4, 3, figsize=(18, 15))
f.suptitle('-- Distribution in mdf_test data -- ', fontsize=17)

for n, col in enumerate(cat_cols):
    sns.countplot(ax=axs[n,0], data=mdf_test, x =col[0])
    axs[n,0].set_title(f"Freq. of each category in {col[0]}")
    axs[n,0].set_xlabel("")
    axs[n,0].set_ylabel("")
    
    sns.countplot(ax=axs[n,1], data=mdf_test, x =col[1])
    axs[n,1].set_title(f"Freq. of each category in {col[1]}")
    axs[n,1].set_xlabel("")
    axs[n,1].set_ylabel("")
    
    sns.countplot(ax=axs[n,2], data=mdf_test, x =col[2])
    axs[n,2].set_title(f"Freq. of each category in {col[2]}")
    axs[n,2].set_xlabel("")
    axs[n,2].set_ylabel("")

f.tight_layout()

#### We are now all good to prepare our data from model building. That includes:--
* Encoding Categorical cols
* Standardizing cell values
##### So before we start that, **let's split the mdf dataset into train & test data,** to prevent *data-leakage.*

##### Saving this cleaned train & test data, for future needs ~~~

In [ ]:
mdf.to_csv("Mushroom train_data__Cleaned.csv")
mdf_test.to_csv("Mushroom test_data__Cleaned.csv")

In [51]:
mdf = pd.read_csv('/kaggle/input/cleaned-data/Mushroom train_data__Cleaned.csv')
mdf_test = pd.read_csv('/kaggle/input/cleaned-data/Mushroom test_data__Cleaned.csv')

mdf.set_index('id', inplace=True)
mdf_test.set_index('id', inplace=True)
mdf.head()

,e_label,cap_diameter,cap_shape,cap_surface,cap_color,does_bruise_or_bleed,gill_attachment,gill_spacing,gill_color,stem_height,stem_width,stem_color,has_ring,ring_type,habitat,season
id,,,,,,,,,,,,,,,,
0,e,8.80,flat,s,other,f,a,close,white,4.51,15.39,white,f,flaring,woods,autumn
1,p,4.51,convex,h,orange,f,a,close,brown,4.79,6.48,other,t,other,woods,winter
2,e,6.94,flat,s,other,f,x,close,white,6.85,9.93,brown,f,flaring,leaves,winter
3,e,3.88,flat,y,gray,f,s,unk,gray,4.16,6.53,white,f,flaring,woods,summer
4,e,5.85,convex,other,white,f,d,unk,white,3.37,8.36,white,f,flaring,grasses,autumn


In [52]:
mdf = mdf[['e_label','cap_diameter','stem_height','stem_width','cap_shape','cap_surface','cap_color','does_bruise_or_bleed','gill_attachment','gill_spacing','gill_color','stem_color','has_ring','ring_type','habitat','season']]
mdf_test = mdf_test[['cap_diameter','stem_height','stem_width','cap_shape','cap_surface','cap_color','does_bruise_or_bleed','gill_attachment','gill_spacing','gill_color','stem_color','has_ring','ring_type','habitat','season']]

In [53]:
test_idx = mdf_test.index

In [22]:
from sklearn.model_selection import train_test_split

mdf_X = mdf.iloc[:,1:]
mdf_Y = mdf['e_label'].map({'e':1, 'p':0}) #a few of our models we will use to predict edibility, 
                                           #requires numerical labels. Hence converting to 1 & 0


mdf_Xtrain, mdf_Xtest, mdf_Ytrain, mdf_Ytest = train_test_split(mdf_X, mdf_Y, test_size=0.3, random_state=25)

In [14]:
mdf_Ytrain

id
1498569    1
2633402    1
2124754    1
854815     0
2570868    0
          ..
928997     0
2517394    1
3067465    1
1192165    0
33810      0
Name: e_label, Length: 1930303, dtype: int64

In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [32]:
ad = mdf_Xtrain.iloc[:1000, :] #a test data, with 100 rows from mdf_Xtrain

# testing to see how many rows occur.
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')
ad_t = encoder.fit_transform(ad.iloc[:,3:])

ad_t.shape

(1000, 44)

In [33]:
#One Hot Encoding -----> after encoding, with drop=first, there will be total 44 encoded columns + the 3 numerical cols

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'),slice(3,15))
],remainder='passthrough')


#MinMax Scaling

scaler = ColumnTransformer([
    ('mmScaler',MinMaxScaler(),slice(0,47))
],remainder='passthrough')

In [34]:
pipe = Pipeline([
    ("one_hot_encoder", encoder),
    ("min_max_Scaler", scaler)
])

pipe.fit(mdf_Xtrain)

Pipeline(steps=[('one_hot_encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  slice(3, 15, None))])),
                ('min_max_Scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('mmScaler', MinMaxScaler(),
                                                  slice(0, 47, None))]))])

In [35]:
mdf_Xtrain = pipe.fit_transform(mdf_Xtrain)
mdf_Xtest = pipe.fit_transform(mdf_Xtest)

In [36]:
print(f"Shape of mdf_Xtrain array: {mdf_Xtrain.shape}")
print(f"Shape of mdf_Xtest array: {mdf_Xtest.shape}")

Shape of mdf_Xtrain array: (1930303, 47)
Shape of mdf_Xtest array: (827273, 47)


In [37]:
print(f"Shape of mdf_Ytrain array: {mdf_Ytrain.shape}")
print(f"Shape of mdf_Ytest array: {mdf_Ytest.shape}")

Shape of mdf_Ytrain array: (1930303,)
Shape of mdf_Ytest array: (827273,)


In [37]:
pipe.named_steps.one_hot_encoder.transformers_[0][1].get_feature_names_out()

array(['cap_shape_convex', 'cap_shape_flat', 'cap_shape_other',
       'cap_shape_sunken', 'cap_surface_g', 'cap_surface_h',
       'cap_surface_other', 'cap_surface_s', 'cap_surface_t',
       'cap_surface_unk', 'cap_surface_y', 'cap_color_gray',
       'cap_color_orange', 'cap_color_other', 'cap_color_red',
       'cap_color_white', 'cap_color_yellow', 'does_bruise_or_bleed_t',
       'gill_attachment_d', 'gill_attachment_e', 'gill_attachment_p',
       'gill_attachment_s', 'gill_attachment_unk', 'gill_attachment_x',
       'gill_spacing_distant', 'gill_spacing_unk', 'gill_color_gray',
       'gill_color_orange', 'gill_color_other', 'gill_color_pink',
       'gill_color_white', 'gill_color_yellow', 'stem_color_other',
       'stem_color_white', 'stem_color_yellow', 'has_ring_t',
       'ring_type_other', 'habitat_leaves', 'habitat_meadows',
       'habitat_other', 'habitat_woods', 'season_spring', 'season_summer',
       'season_winter'], dtype=object)

#### Let's save the Pipiline as well, using **Pickle library.**

In [38]:
import pickle

#creating a pickle file of my pipe.
# pickle.dump(pipe, open('tx_pipe.pkl', 'wb'))

# loading pickled pipeline 
tx_pipe = pickle.load(open('/kaggle/input/tx-pipeline/tx_pipe.pkl', 'rb'))

In [38]:
# original test data
mdf_test = pipe.fit_transform(mdf_test)
print(f"Shape of mdf_test array: {mdf_test.shape}")

Shape of mdf_test array: (1838614, 47)


### **Model Building:**

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

In [42]:
lr = LogisticRegression(random_state=2, solver='newton-cholesky')
dt = DecisionTreeClassifier(random_state=3, max_features=25)
rf = RandomForestClassifier(random_state=0, n_estimators=25, max_features = 20)
gnb = GaussianNB()
gbc = GradientBoostingClassifier(random_state=1, n_estimators=25)
xgb = XGBClassifier()

In [43]:
import time

model_report = pd.DataFrame(columns = ["accuracy", "precision", "recall", "f1", "Time taken"])

models = [lr,dt,rf,gnb,gbc,xgb]
model_names = ["Logistic Regression", "Decision Tree Clf", "Random Forest Clf", "Gaussian Naive Bayes", "GB Clf", "XGB Clf"]

for i, model in enumerate(models):
    
    start = time.time()
    
    model.fit(mdf_Xtrain,mdf_Ytrain)
    print(f"'{model_names[i]}' is ready to predict:~ ")
    mdf_Ypred = model.predict(mdf_Xtest)
    
    time_taken = time.time()-start
    
    acc = accuracy_score(mdf_Ytest, mdf_Ypred)
    p = precision_score(mdf_Ytest, mdf_Ypred, average='weighted')
    r = recall_score(mdf_Ytest, mdf_Ypred, average='weighted')
    f = f1_score(mdf_Ytest, mdf_Ypred, average='weighted')
    print("Classification Reports:-\n", classification_report(mdf_Ytest, mdf_Ypred))
    print("\nConfusion Matrix:-\n", confusion_matrix(mdf_Ytest, mdf_Ypred))
    
    model_report = pd.concat([model_report, pd.DataFrame([{'accuracy':acc, 'precision':p, 'recall':r, 'f1':f, "Time taken":time_taken}])])
    print(f"Report of '{model_names[i]}' is recorded.", "\n========================")

model_report["model"] = model_names
model_report = model_report[["model","accuracy", "precision", "recall", "f1", "Time taken"]]

Logistic Regression is ready to predict:~ 
Classification Reports:-
               precision    recall  f1-score   support

           0       0.79      0.77      0.78    449924
           1       0.73      0.75      0.74    377349

    accuracy                           0.76    827273
   macro avg       0.76      0.76      0.76    827273
weighted avg       0.76      0.76      0.76    827273


Confusion Matrix:-
 [[346601 103323]
 [ 94794 282555]]
Report of Logistic Regression is recorded. 
Decision Tree Clf is ready to predict:~ 
Classification Reports:-
               precision    recall  f1-score   support

           0       0.98      0.98      0.98    449924
           1       0.98      0.98      0.98    377349

    accuracy                           0.98    827273
   macro avg       0.98      0.98      0.98    827273
weighted avg       0.98      0.98      0.98    827273


Confusion Matrix:-
 [[442699   7225]
 [  7160 370189]]
Report of Decision Tree Clf is recorded. 
Random Fores

In [44]:
model_report

,model,accuracy,precision,recall,f1,Time taken
0,Logistic Regression,0.760518,0.761064,0.760518,0.760710,7.219208
0,Decision Tree Clf,0.982612,0.982612,0.982612,0.982612,37.780317
0,Random Forest Clf,0.991048,0.991052,0.991048,0.991049,468.157669
0,Gaussian Naive Bayes,0.707031,0.708834,0.707031,0.707492,2.469409
0,GB Clf,0.866454,0.871475,0.866454,0.865077,190.281051
0,XGB Clf,0.990362,0.990366,0.990362,0.990363,29.083586


#### //- SUMMARY from above:
* **Random Classifier has the best accuracy among all.** Generally RF performs the best, when models are used with not much parameter tweakings. The problem with Random forest is that, it took the maximum time among all. And if we had used the default value of n_estimators, i.e. 100, it might have taken a long span too. 
* **XGB Classifier though takes very less time, and misses the most accurate result by a whisker.** And the same is for Decision tree as well. 
* **Gradiant Boosting Classifier took a while to fit and then predict**, and it had an OKAY accuracy. Paramter tunning might improve its efficiency.
* **LR & GNB** we so quick to predict, and **had comparatively low accuracy** as well.

* I had tried to build **KNN and SVM** earlier, and it seemed like those **would take ages to fit & predict.** Those literally ran 30-40 minutes, and the fit was still in progress before I canceled and removed those two models. 
#### 
#### Let's try out now with XGB, as it was very fast. Later we can do parameter tuning on RF, XGB, DT & GB, and try and see if we can go any further than .991

In [45]:
#Predicting output for Original test data
mdf_pred = xgb.predict(mdf_test)

In [48]:
display(mdf_pred)
mdf_pred.shape

array([1, 0, 0, ..., 0, 1, 1])

(1838614,)

##### Let's create a dataframe, and decode 1s and 0s back to e_label:

In [56]:
my_submission = pd.DataFrame({'id':test_idx, 'class':mdf_pred})
my_submission['class'] = my_submission['class'].map({1:'e', 0:'p'})
my_submission.to_csv("My Submission __1.csv")